In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Actividad 1: Análisis de Ventas con Gráficos Interactivos
##Contexto:
Durante esta semana de tu pasantía, tu primera tarea consiste en analizar datos de ventas. Silvia, Project Manager y Data Scientist, requiere un análisis detallado del rendimiento de las ventas de los productos principales.
##Objetivos:
* Crear visualizaciones interactivas para analizar las ventas de diferentes productos.
* Identificar tendencias y patrones clave en los datos de ventas.
* Presentar los hallazgos de manera clara y concisa utilizando Plotly.

##Ejercicio práctico:
1. Se solicita la creación de gráficos interactivos que permitan visualizar la evolución de las ventas a lo largo del tiempo, comparar el rendimiento de los diferentes productos y filtrar por región.
2. Desarrollo:
  ◦ Cargar el dataset de ventas
  ◦ Utilizar Plotly para crear un gráfico de líneas que muestre la evolución de las ventas a lo largo del tiempo para cada producto.
  ◦ Implementar interactividad: Permitir que los usuarios puedan pasar el cursor sobre las líneas para visualizar los valores exactos de ventas en cada fecha.
  ◦ Crear un gráfico de barras que compare las ventas totales de cada producto.
  ◦ Implementar filtros para que los usuarios puedan seleccionar una región específica y visualizar las ventas únicamente para esa región.
3. Presentación: Preparar una presentación concisa que incluya las visualizaciones y una explicación de los hallazgos principales.

##Dataset
* [ventas_por_region.csv](https://drive.google.com/file/d/1UEURMRy6ep9RUXfvkjwHvzTboqU4Hzjk/view?usp=drive_link)

##Por qué importa esto en SynthData?
En SynthData, el análisis de ventas es fundamental para tomar decisiones informadas sobre el desarrollo de productos, las estrategias de marketing y la expansión a nuevas regiones. Una visualización interactiva permite al equipo explorar los datos en detalle, identificar oportunidades de crecimiento y anticipar posibles desafíos. El desarrollo de estas habilidades contribuye a la formación de analistas de datos competentes.

In [9]:
# LIBRERÍA DASH tiene módulo deprecados. Nueva solución con plotly.express y widgets
# -----------------------------------------
# Análisis de Ventas con Plotly Interactivo
# -----------------------------------------

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Crear el dataset base
df = pd.read_csv("/content/drive/MyDrive/datasets/ventas_por_region.csv")



In [10]:
# -----------------------------------------------------
# 0. Exploración inicial y preparación
# -----------------------------------------------------
print(df.info())
print(df.describe())
print(df.head())

# Tipo de dato
df['fecha'] = pd.to_datetime(df['fecha'])
df['ventas'] = df['ventas'].str.replace('$','').astype('float')

# Agrupación mensual
df['mes'] = df['fecha'].dt.to_period('M').dt.to_timestamp()
df_mensual = df.groupby(['mes', 'producto'])['ventas'].sum().reset_index()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fecha     1000 non-null   object
 1   producto  1000 non-null   object
 2   region    1000 non-null   object
 3   ventas    1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB
None
             fecha  producto region    ventas
count         1000      1000   1000      1000
unique         337         5      5      1000
top     2024-10-22  Notebook  Oeste  $6302.93
freq             8       206    209         1
        fecha     producto  region     ventas
0  2025-03-24  Auriculares    Este   $9984.02
1  2024-11-04     Smart TV    Este   $8723.07
2  2025-05-14  Auriculares   Oeste  $11326.45
3  2025-04-21     Smart TV  Centro   $7404.36
4  2025-05-18   Smartphone   Oeste   $9358.53


In [11]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   fecha     1000 non-null   datetime64[ns]
 1   producto  1000 non-null   object        
 2   region    1000 non-null   object        
 3   ventas    1000 non-null   float64       
 4   mes       1000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(2)
memory usage: 39.2+ KB
None
                     fecha        ventas                            mes
count                 1000   1000.000000                           1000
mean   2024-12-02 14:02:24   8700.787130  2024-11-18 02:49:55.199999744
min    2024-06-10 00:00:00   4568.520000            2024-06-01 00:00:00
25%    2024-09-04 00:00:00   6759.317500            2024-09-01 00:00:00
50%    2024-11-27 00:00:00   8782.180000            2024-11-01 00:00:00
75%    2025-03-04 00:00:00  10794.950000            2025-03-01 0

In [12]:

# -----------------------------------------
# 1. Evolución mensual de ventas por producto
# -----------------------------------------

df_mensual = df.groupby(['mes', 'producto'])['ventas'].sum().reset_index()

fig_line = px.line(
    df_mensual,
    x="mes",
    y="ventas",
    color="producto",
    title="Evolución Mensual de Ventas por Producto"
)
fig_line.update_layout(xaxis_title="Mes", yaxis_title="Ventas")
fig_line.show()

# -----------------------------------------
# 2. Ventas totales por producto (barras)
# -----------------------------------------

df_totales = df.groupby("producto")["ventas"].sum().reset_index()

fig_bar = px.bar(
    df_totales,
    x="producto",
    y="ventas",
    title="Ventas Totales por Producto",
    color="producto"
)
fig_bar.show()

# -----------------------------------------
# 3. Interactividad: Filtro por región
# -----------------------------------------

# Widget de selección de regiones
regiones = df['region'].unique().tolist()
selector = widgets.SelectMultiple(
    options=regiones,
    value=[regiones[0]],
    description='Región',
    disabled=False
)

# Función que agrupa por mes y actualiza gráfico según región
def mostrar_grafico(regiones_seleccionadas):
    df_filtrado = df[df['region'].isin(regiones_seleccionadas)].copy()
    df_agrupado = df_filtrado.groupby(['mes', 'producto'])['ventas'].sum().reset_index()

    fig = px.line(
        df_agrupado,
        x="mes",
        y="ventas",
        color="producto",
        title=f"Evolución Mensual de Ventas por Producto (Regiones: {', '.join(regiones_seleccionadas)})"
    )
    fig.update_layout(xaxis_title="Mes", yaxis_title="Ventas",width=1200)
    fig.show()

# Mostrar selector interactivo
widgets.interact(mostrar_grafico, regiones_seleccionadas=selector)

interactive(children=(SelectMultiple(description='Región', index=(0,), options=('Este', 'Oeste', 'Centro', 'Su…

<function __main__.mostrar_grafico(regiones_seleccionadas)>

#Actividad 2: Dashboard de Redes Sociales
##Contexto:
Sabrina, la Data Engineer, ha implementado un sistema para recopilar datos de redes sociales. Se requiere el diseño de un dashboard para visualizar y analizar estos datos.
##Objetivos:
* Crear un dashboard interactivo para mostrar las métricas clave de redes sociales.
* Permitir a los usuarios filtrar los datos por plataforma y período de tiempo.
* Diseñar un dashboard visualmente atractivo y fácil de usar.

##Ejercicio práctico:
1. Se solicita la creación de un dashboard que muestre la cantidad de seguidores, "me gusta" y comentarios en las redes sociales de la empresa. Es importante poder visualizar la evolución de estas métricas a lo largo del tiempo y filtrar por plataforma para identificar áreas de mayor impacto.
2. Desarrollo:
* Cargar el dataset de redes sociales
* Utilizar Dash y Plotly para crear un dashboard con los siguientes elementos:
  * Un gráfico de líneas que muestre la evolución de la cantidad de seguidores a lo largo del tiempo para cada plataforma (Facebook, Twitter, Instagram, etc.).
  * Un gráfico de barras que compare la cantidad de "me gusta" y comentarios en cada plataforma.
  * Filtros que permitan a los usuarios seleccionar una o varias plataformas y un período de tiempo específico.
  * Títulos claros y concisos para cada gráfico.
          
##Dataset

* [rr_ss.csv](https://drive.google.com/file/d/1jThaa9yHF4C0KnU_qiL463w_qBnJw_oL/view?usp=drive_link)


##¿Por qué importa esto en SynthData?
El monitoreo de las redes sociales es crucial para comprender el impacto de las campañas de marketing, identificar tendencias emergentes y mejorar la interacción con los clientes. Un dashboard interactivo permite al equipo tomar decisiones basadas en datos y optimizar sus estrategias en tiempo real. Esta actividad proporciona una visión integral del ciclo de vida de los datos, desde la recolección hasta la visualización y el análisis.

In [13]:

# --------------------------------------------------
# Dashboard Interactivo de Redes Sociales con Plotly
# --------------------------------------------------

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# ----------------------------------------
# 1. Crear el dataset y preparar los datos
# ----------------------------------------
df = pd.read_csv("/content/drive/MyDrive/datasets/rrss.csv")

df['fecha'] = pd.to_datetime(df['fecha'])
df['mes'] = df['fecha'].dt.to_period('M').dt.to_timestamp()

# ---------------------------------------------
# 2. Gráfico de evolución mensual de seguidores
# ---------------------------------------------
seguidores_mensuales = df.groupby(['mes', 'plataforma'])['seguidores'].mean().reset_index()

fig_seguidores = px.line(
    seguidores_mensuales,
    x="mes",
    y="seguidores",
    color="plataforma",
    title="Evolución Mensual de Seguidores por Plataforma"
)
fig_seguidores.update_layout(width=1000, xaxis_title="Mes", yaxis_title="Seguidores")
fig_seguidores.show()

# ------------------------------------------------------------
# 3. Gráfico comparativo de likes y comentarios por plataforma
# ------------------------------------------------------------
interaccion_total = df.groupby('plataforma')[['likes', 'comentarios']].sum().reset_index()
interaccion_melt = interaccion_total.melt(id_vars='plataforma', var_name='métrica', value_name='cantidad')

fig_interaccion = px.bar(
    interaccion_melt,
    x="plataforma",
    y="cantidad",
    color="métrica",
    barmode="group",
    title="Comparación Total de Likes y Comentarios por Plataforma"
)
fig_interaccion.update_layout(width=1000)
fig_interaccion.show()

# -----------------------------------------------
# 4. Filtros interactivos por plataforma y fecha
# -----------------------------------------------

# Widgets
plataformas = df['plataforma'].unique().tolist()
selector_plataforma = widgets.SelectMultiple(
    options=plataformas,
    value=[plataformas[0]],
    description='Plataforma'
)

selector_fecha = widgets.SelectionRangeSlider(
    options=sorted(df['fecha'].dt.date.unique()),
    index=(0, len(df['fecha'].unique()) - 1),
    description='Fechas'
)

# Función interactiva
def actualizar_dashboard(plataformas_seleccionadas, fechas):
    fechas = pd.to_datetime(list(fechas))
    df_filtrado = df[
        (df['plataforma'].isin(plataformas_seleccionadas)) &
        (df['fecha'] >= fechas[0]) & (df['fecha'] <= fechas[1])
    ]
    df_filtrado['mes'] = df_filtrado['fecha'].dt.to_period('M').dt.to_timestamp()
    df_agrupado = df_filtrado.groupby(['mes', 'plataforma'])['seguidores'].mean().reset_index()

    fig = px.line(
        df_agrupado,
        x="mes",
        y="seguidores",
        color="plataforma",
        title=f"Evolución de Seguidores (Filtrado)"
    )
    fig.update_layout(width=1000, xaxis_title="Mes", yaxis_title="Seguidores")
    fig.show()

    # Interfaz
widgets.interact(
    actualizar_dashboard,
    plataformas_seleccionadas=selector_plataforma,
    fechas=selector_fecha
)

interactive(children=(SelectMultiple(description='Plataforma', index=(0,), options=('TikTok', 'YouTube', 'Face…

<function __main__.actualizar_dashboard(plataformas_seleccionadas, fechas)>